In [1]:
# load scalp example
import trimesh
import numpy as np
import pyvista as pv
from electrodes_positions.utils.mesh_utils import faces_to_pyvista

# scalp surface is from subject 16, session 2 in
# Telesford, Q.K., Gonzalez-Moreira, E., Xu, T. et al. An open-access dataset of naturalistic viewing using simultaneous EEG-fMRI. Sci Data 10, 554 (2023). https://doi.org/10.1038/s41597-023-02458-8

mesh = trimesh.load('outer_skin.stl')
vertices = np.array(mesh.vertices)
faces = np.array(mesh.faces)

### Anatomical montage for comparison

In [2]:
from electrodes_positions.utils.point_picking import pick_fiducials

# pick fiducials using a GUI
picked_points = pick_fiducials(vertices, faces)

Widget(value='<iframe src="http://localhost:36151/index.html?ui=P_0x7fbc3952fd70_0&reconnect=auto" class="pyvi…

In [3]:
from electrodes_positions.utils.point_picking import project_fid_on_mesh

# project the fiducials on the mesh vertices
(RPA, LPA, NAS, IN), (RPA_idx, LPA_idx, NAS_idx, IN_idx) = project_fid_on_mesh(picked_points, vertices, return_positions = True, return_indices=True)

In [4]:
# creates a standard montage according to the desired system
# see available montages with print(electrodes_positions.montages.available_montages)
from electrodes_positions.montages import create_standard_montage

all_landmarks = create_standard_montage(vertices, faces, fiducials = (RPA_idx, LPA_idx, NAS_idx, IN_idx), system = '10-10', return_indices = False)

# Basic three points coregistration
Usually good with real positions measured on head, a transformation is computed to align the fiducial points. Downside is that it requires you to have both the real and the mesh position of the fiducials (which are not correct in this dataset).

In [32]:
# real positions measured on patient's head
with open('ch_pos.sfp', 'r') as f:
    real_landmarks = f.readlines()
real_landmarks = dict(map(lambda x: (x.strip().split('\t')[0], np.array(x.strip().split('\t')[1:], dtype = float)), real_landmarks))

In [33]:
from electrodes_positions.coregistration import transform_fiducials

# aligns the positions with the head using only the fiducials
new_real_landmarks = transform_fiducials(real_landmarks, (RPA, LPA, NAS, IN), scale_y = False, shear_y = False)

In [34]:
from electrodes_positions.coregistration import project_electrodes_on_mesh

# projects the positions on the scalp
projected_new_real_landmarks = project_electrodes_on_mesh(new_real_landmarks, vertices, faces)

In [ ]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(np.array(list(new_real_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'yellow')
plotter.add_points(np.array(list(projected_new_real_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'blue')
plotter.add_points(np.array(list(all_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'green')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

# Automatic coregistration
This uses an iterative closest point technique to coregister the positions to the head.

It is good practice to start with a rough registration first (even hand made, if possible).

In [ ]:
# real positions measured on patient's head
with open('ch_pos.sfp', 'r') as f:
    real_landmarks = f.readlines()
real_landmarks = dict(map(lambda x: (x.strip().split('\t')[0], np.array(x.strip().split('\t')[1:], dtype = float)), real_landmarks))

In [ ]:
from electrodes_positions.coregistration import transform_fiducials

# aligns the positions with the head using only the fiducials
new_real_landmarks = transform_fiducials(real_landmarks, (RPA, LPA, NAS, IN), scale_y = False, shear_y = False)

In [40]:
from electrodes_positions.coregistration import coregister_to_mesh

# coregisters the position with the scalp
coregistered_new_real_landmarks = coregister_to_mesh(vertices, faces, new_real_landmarks, DoF = 7, projection = 'approximate', project_result = True)

In [41]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(np.array(list(coregistered_new_real_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'blue')
plotter.add_points(np.array(list(all_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'green')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:41327/index.html?ui=P_0x7fb48473f8c0_6&reconnect=auto" class="pyvi…

# Affine automatic coregistration
Useful if points belong to a different head, so scaling and shears are needed to align the positions to the head.

In [42]:
# positions computed on a generic standard head
with open('standard_10-10.sfp', 'r') as f:
    standard_landmarks = f.readlines()
standard_landmarks = dict(map(lambda x: (x.strip().split('\t')[0], np.array(x.strip().split('\t')[1:], dtype = float)), standard_landmarks))

In [43]:
from electrodes_positions.coregistration import transform_fiducials

# aligns the positions with the head using only the fiducials
new_standard_landmarks = transform_fiducials(standard_landmarks, (RPA, LPA, NAS, IN), scale_y = False, shear_y = False)

In [ ]:
from electrodes_positions.coregistration import coregister_to_mesh

# coregisters the position with the scalp
coregistered_new_standard_landmarks = coregister_to_mesh(vertices, faces, new_standard_landmarks, DoF = 12, projection = 'approximate', project_result = True)

In [45]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(np.array(list(coregistered_new_standard_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'blue')
plotter.add_points(np.array(list(all_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'green')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:41327/index.html?ui=P_0x7fb4bf2d6ed0_7&reconnect=auto" class="pyvi…

# 6, 7, 9, 12 degrees of freedom
You can limit the operations allowed on the positions for the coregistration:
* DoF = 6:  rigid transformations
* DoF = 7:  rigid transformations with a global scaling factor
* DoF = 9:  rotations, traslations, and scalings
* DoF = 12:  affine transformations (i.e. rotations, traslations, scalings, and shears)

In [10]:
# positions computed on a generic standard head
with open('standard_10-10.sfp', 'r') as f:
    standard_landmarks = f.readlines()
standard_landmarks = dict(map(lambda x: (x.strip().split('\t')[0], np.array(x.strip().split('\t')[1:], dtype = float)), standard_landmarks))

In [11]:
from electrodes_positions.coregistration import transform_fiducials

# aligns the positions with the head using only the fiducials
new_standard_landmarks = transform_fiducials(standard_landmarks, (RPA, LPA, NAS, IN), scale_y = False, shear_y = False)

In [12]:
from electrodes_positions.coregistration import coregister_to_mesh

# coregisters the position with the scalp
coregistered_6_new_standard_landmarks = coregister_to_mesh(vertices, faces, new_standard_landmarks, DoF = 6, projection = 'approximate', project_result = True)
coregistered_7_new_standard_landmarks = coregister_to_mesh(vertices, faces, new_standard_landmarks, DoF = 7, projection = 'approximate', project_result = True)
coregistered_9_new_standard_landmarks = coregister_to_mesh(vertices, faces, new_standard_landmarks, DoF = 9, projection = 'approximate', project_result = True)
coregistered_12_new_standard_landmarks = coregister_to_mesh(vertices, faces, new_standard_landmarks, DoF = 12, projection = 'approximate', project_result = True)

In [13]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(np.array(list(coregistered_6_new_standard_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_points(np.array(list(coregistered_7_new_standard_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'white')
plotter.add_points(np.array(list(coregistered_9_new_standard_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'black')
plotter.add_points(np.array(list(coregistered_12_new_standard_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'blue')
plotter.add_points(np.array(list(all_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'green')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:36151/index.html?ui=P_0x7fbc2b435670_1&reconnect=auto" class="pyvi…

# "exact" coregistration
This is an improved version of the algorithm which uses exact projection on the mesh faces to run the ICP. It's much slower, but can lead to better results.

The two methods are basically the same if the mesh is fine enough. In this specific case they produce the same solution.

In [14]:
# real positions measured on patient's head
with open('ch_pos.sfp', 'r') as f:
    real_landmarks = f.readlines()
real_landmarks = dict(map(lambda x: (x.strip().split('\t')[0], np.array(x.strip().split('\t')[1:], dtype = float)), real_landmarks))

In [15]:
from electrodes_positions.coregistration import transform_fiducials

# aligns the positions with the head using only the fiducials
new_real_landmarks = transform_fiducials(real_landmarks, (RPA, LPA, NAS, IN), scale_y = False, shear_y = False)

In [ ]:
from electrodes_positions.coregistration import coregister_to_mesh

# coregisters the position with the scalp
exact_coregistered_new_real_landmarks = coregister_to_mesh(vertices, faces, new_real_landmarks, DoF = 7, projection = 'exact', project_result = True)

In [ ]:
from electrodes_positions.coregistration import coregister_to_mesh

# coregisters the position with the scalp
approx_coregistered_new_real_landmarks = coregister_to_mesh(vertices, faces, new_real_landmarks, DoF = 7, projection = 'exact', project_result = True)

In [9]:
mesh = pv.PolyData(vertices, faces_to_pyvista(faces))

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='red')
plotter.add_points(np.array(list(exact_coregistered_new_real_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'blue')
plotter.add_points(np.array(list(approx_coregistered_new_real_landmarks.values())), render_points_as_spheres = True, point_size = 10, color = 'yellow')
plotter.add_points(np.array(list(all_landmarks.values())), render_points_as_spheres = True, point_size = 5, color = 'green')
plotter.add_point_labels([RPA, LPA, NAS, IN], ['RPA', 'LPA', 'NAS', 'IN'], render_points_as_spheres = True, point_size = 15, point_color = 'green')
plotter.show()

Widget(value='<iframe src="http://localhost:36151/index.html?ui=P_0x7fbc39785e80_0&reconnect=auto" class="pyvi…